# Important links

Combining two pretrained separate models side by side: https://discuss.pytorch.org/t/combining-trained-models-in-pytorch/28383/7?u=nagabhushansn95

combine two models side by side and train (using graph neural networks): https://datascience.stackexchange.com/questions/117110/is-it-possible-to-combine-models-in-pytorch-and-pytorch-geometric

Combining two separate models one after the other: https://stackoverflow.com/questions/65216411/how-to-concatenate-2-pytorch-models-and-make-the-first-one-non-trainable-in-pyto

In [39]:
import loss_fn

ModuleNotFoundError: No module named 'loss_fn'

In [1]:
import torch
import torch.nn as nn
import numpy as np
import os

In [ ]:
'''
to do: train one to get scale (std)
    get the intended std outputs
    map each to an input
    train.

train both and get the output into a matrix.
'''

In [2]:
ALL_PHONEMES =['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay', 'b', 'bcl', 'ch', 'd', 'dcl', 'dh', 'dx', 'eh', 'el', 'em', 'en', 'eng', 'epi', 'er', 'ey', 'f', 'g', 'gcl', 'h#', 'hh', 'hv', 'ih', 'ix', 'iy', 'jh', 'k', 'kcl', 'l', 'm', 'n', 'ng', 'nx', 'ow', 'oy', 'p', 'pau', 'pcl', 'q', 'r', 's', 'sh', 't', 'tcl', 'th', 'uh', 'uw', 'ux', 'v', 'w', 'y', 'z', 'zh']

In [51]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size, activation):
        super(MLP, self).__init__()
        self.sequential = nn.Sequential()
        self.sequential.add_module("dense1", nn.Linear(input_size, output_size))
        self.sequential.add_module("act1", activation())

    def forward(self, x):
        h = self.sequential(x)
#         print(h.shape)
        return h

In [3]:
predictions_folder = "saved_variables/average_and_std_phoneme_feature/"

In [4]:
# load dataset

phonemes = []
scales = []
shifts = []

for indx in range(len(ALL_PHONEMES)):
    phoneme_tensor = np.zeros(len(ALL_PHONEMES))
    phoneme_tensor[indx] = 1
    scale_file = predictions_folder+ALL_PHONEMES[indx]+"_std.txt"
    shift_file = predictions_folder+ALL_PHONEMES[indx]+"_average.txt"
    if os.path.isfile(scale_file):
        scales.append(np.loadtxt(scale_file)[:65])
        shifts.append(np.loadtxt(shift_file)[:65])
        phonemes.append(phoneme_tensor)
#         print(ALL_PHONEMES[indx])
        
phonemes = np.array(phonemes)
scales = np.array(scales)
shifts = np.array(shifts)

phonemes = torch.from_numpy(phonemes).float()
scales = torch.from_numpy(scales).float()
shifts = torch.from_numpy(shifts).float()

In [ ]:
num_epochs = 1000

In [ ]:
scale_model = MLP(61, 65, nn.ReLU)
optimizer_scales = torch.optim.Adam(scale_model.parameters(), lr=0.03)
loss_fn = nn.MSELoss()

losses = []

for epoch in range(num_epochs):
    scales_pred = scale_model(phonemes)
    loss_scale = loss_fn(scales_pred,scales)
    losses.append(loss_scale)
    print(f"epoch: {epoch}, loss: {loss_scale}")
    optimizer_scales.zero_grad()
    loss_scale.backward()
    optimizer_scales.step()
#     if len(losses)>10:
#         if losses[-1]>=losses[-10] or losses[-1]>=losses[-5]:
#             print("detecting minimum validation loss. stopping early")
#             break

In [18]:
# TODO: edit this to make relevant to saving both models
torch.save({"model":scale_model.state_dict(), "optimizer":optimizer_scales.state_dict()}, "scales_model")

In [75]:
shift_model = MLP(61, 65, nn.LeakyReLU)
optimizer_shifts = torch.optim.SGD(shift_model.parameters(), lr=1500)
loss_fn = nn.MSELoss()

losses = []

for epoch in range(num_epochs):
    shifts_pred = shift_model(phonemes)
    loss_shift = loss_fn(shifts_pred,shifts)
    losses.append(loss_shift)
    print(f"epoch: {epoch}, loss: {loss_shift}")
    optimizer_shifts.zero_grad()
    loss_shift.backward()
    optimizer_shifts.step()
#     if len(losses)>10:
#         if losses[-1]>=losses[-10] or losses[-1]>=losses[-5]:
#             print("detecting minimum validation loss. stopping early")
#             break

epoch: 0, loss: 50.96644592285156
epoch: 1, loss: 31.800708770751953
epoch: 2, loss: 31.497467041015625
epoch: 3, loss: 31.215972900390625
epoch: 4, loss: 30.937118530273438
epoch: 5, loss: 30.660869598388672
epoch: 6, loss: 30.387210845947266
epoch: 7, loss: 30.11610221862793
epoch: 8, loss: 29.847532272338867
epoch: 9, loss: 29.58147430419922
epoch: 10, loss: 29.317909240722656
epoch: 11, loss: 29.05680274963379
epoch: 12, loss: 28.79814338684082
epoch: 13, loss: 28.541902542114258
epoch: 14, loss: 28.28805160522461
epoch: 15, loss: 28.036579132080078
epoch: 16, loss: 27.787460327148438
epoch: 17, loss: 27.540666580200195
epoch: 18, loss: 27.296184539794922
epoch: 19, loss: 27.053987503051758
epoch: 20, loss: 26.814054489135742
epoch: 21, loss: 26.576372146606445
epoch: 22, loss: 26.340906143188477
epoch: 23, loss: 26.10764503479004
epoch: 24, loss: 25.876562118530273
epoch: 25, loss: 25.64763832092285
epoch: 26, loss: 25.42084503173828
epoch: 27, loss: 25.19183921813965
epoch: 28, l

epoch: 275, loss: 3.7447731494903564
epoch: 276, loss: 3.7225167751312256
epoch: 277, loss: 3.700467586517334
epoch: 278, loss: 3.6786224842071533
epoch: 279, loss: 3.656980037689209
epoch: 280, loss: 3.635538101196289
epoch: 281, loss: 3.6142945289611816
epoch: 282, loss: 3.5932483673095703
epoch: 283, loss: 3.572396993637085
epoch: 284, loss: 3.551739454269409
epoch: 285, loss: 3.5312724113464355
epoch: 286, loss: 3.510995626449585
epoch: 287, loss: 3.490906238555908
epoch: 288, loss: 3.471003532409668
epoch: 289, loss: 3.451284885406494
epoch: 290, loss: 3.431749105453491
epoch: 291, loss: 3.412393569946289
epoch: 292, loss: 3.3932182788848877
epoch: 293, loss: 3.3742194175720215
epoch: 294, loss: 3.3553969860076904
epoch: 295, loss: 3.336749315261841
epoch: 296, loss: 3.3182737827301025
epoch: 297, loss: 3.299968957901001
epoch: 298, loss: 3.2818338871002197
epoch: 299, loss: 3.263866662979126
epoch: 300, loss: 3.246065378189087
epoch: 301, loss: 3.228429079055786
epoch: 302, loss:

epoch: 649, loss: 1.3930391073226929
epoch: 650, loss: 1.392095923423767
epoch: 651, loss: 1.3911596536636353
epoch: 652, loss: 1.390230655670166
epoch: 653, loss: 1.3893083333969116
epoch: 654, loss: 1.3883928060531616
epoch: 655, loss: 1.387483835220337
epoch: 656, loss: 1.386581540107727
epoch: 657, loss: 1.3856861591339111
epoch: 658, loss: 1.3847970962524414
epoch: 659, loss: 1.3839147090911865
epoch: 660, loss: 1.383038878440857
epoch: 661, loss: 1.3821691274642944
epoch: 662, loss: 1.3813056945800781
epoch: 663, loss: 1.3804484605789185
epoch: 664, loss: 1.379597544670105
epoch: 665, loss: 1.3787529468536377
epoch: 666, loss: 1.3778150081634521
epoch: 667, loss: 1.37073814868927
epoch: 668, loss: 1.6953089237213135
epoch: 669, loss: 1.4184107780456543
epoch: 670, loss: 1.4172008037567139
epoch: 671, loss: 1.4160003662109375
epoch: 672, loss: 1.414809226989746
epoch: 673, loss: 1.4136276245117188
epoch: 674, loss: 1.4124552011489868
epoch: 675, loss: 1.4112919569015503
epoch: 676

In [71]:
# former method: directly training without wrapping in class

losses = []
shift_model = nn.Sequential()
shift_model.add_module("dense1", nn.Linear(61, 65))
shift_model.add_module("act1", nn.LeakyReLU())

loss_fn = nn.MSELoss()

optimizer_shifts = torch.optim.SGD(shift_model.parameters(), lr=1500)

for epoch in range(num_epochs):
    shifts_pred = shift_model(phonemes)
    loss_shift = loss_fn(shifts_pred,shifts)
    losses.append(loss_shift)
    print(f"epoch: {epoch}, loss: {loss_shift}")
    optimizer_shifts.zero_grad()
    loss_shift.backward()
    optimizer_shifts.step()
    
#     if len(losses)>10:
#         if losses[-1]-losses[-5]>=0:
#             print("detecting minimum validation loss. stopping early")
#             break

epoch: 0, loss: 50.91822052001953
epoch: 1, loss: 34.56864929199219
epoch: 2, loss: 33.04054260253906
epoch: 3, loss: 32.74462890625
epoch: 4, loss: 32.45148849487305
epoch: 5, loss: 32.16109085083008
epoch: 6, loss: 31.87339973449707
epoch: 7, loss: 31.588050842285156
epoch: 8, loss: 32.95539855957031
epoch: 9, loss: 31.412677764892578
epoch: 10, loss: 31.131986618041992
epoch: 11, loss: 30.85392189025879
epoch: 12, loss: 30.57845115661621
epoch: 13, loss: 30.305564880371094
epoch: 14, loss: 30.035228729248047
epoch: 15, loss: 29.767417907714844
epoch: 16, loss: 29.50211524963379
epoch: 17, loss: 29.23929214477539
epoch: 18, loss: 28.97892951965332
epoch: 19, loss: 28.72100067138672
epoch: 20, loss: 28.465484619140625
epoch: 21, loss: 28.212356567382812
epoch: 22, loss: 27.96159553527832
epoch: 23, loss: 27.713178634643555
epoch: 24, loss: 27.467086791992188
epoch: 25, loss: 27.223295211791992
epoch: 26, loss: 26.981788635253906
epoch: 27, loss: 26.74253273010254
epoch: 28, loss: 26.5

epoch: 397, loss: 2.145137310028076
epoch: 398, loss: 2.1376326084136963
epoch: 399, loss: 2.1301958560943604
epoch: 400, loss: 2.12282657623291
epoch: 401, loss: 2.115525007247925
epoch: 402, loss: 2.1082892417907715
epoch: 403, loss: 2.1011202335357666
epoch: 404, loss: 2.094015598297119
epoch: 405, loss: 2.086975574493408
epoch: 406, loss: 2.080000162124634
epoch: 407, loss: 2.073087215423584
epoch: 408, loss: 2.066237688064575
epoch: 409, loss: 2.059450149536133
epoch: 410, loss: 2.0527243614196777
epoch: 411, loss: 2.0460598468780518
epoch: 412, loss: 2.0394556522369385
epoch: 413, loss: 2.0329113006591797
epoch: 414, loss: 2.026426076889038
epoch: 415, loss: 2.0199999809265137
epoch: 416, loss: 2.013632297515869
epoch: 417, loss: 2.007322072982788
epoch: 418, loss: 2.0010688304901123
epoch: 419, loss: 1.9948724508285522
epoch: 420, loss: 1.9887322187423706
epoch: 421, loss: 1.9826472997665405
epoch: 422, loss: 1.976617693901062
epoch: 423, loss: 1.9706425666809082
epoch: 424, los

epoch: 800, loss: 1.31748366355896
epoch: 801, loss: 1.3168535232543945
epoch: 802, loss: 1.308613657951355
epoch: 803, loss: 1.316056489944458
epoch: 804, loss: 1.315571665763855
epoch: 805, loss: 1.3150861263275146
epoch: 806, loss: 1.307017207145691
epoch: 807, loss: 1.3153187036514282
epoch: 808, loss: 1.3148338794708252
epoch: 809, loss: 1.314352035522461
epoch: 810, loss: 1.3138726949691772
epoch: 811, loss: 1.3133959770202637
epoch: 812, loss: 1.3129221200942993
epoch: 813, loss: 1.312450885772705
epoch: 814, loss: 1.311982274055481
epoch: 815, loss: 1.311516284942627
epoch: 816, loss: 1.3110527992248535
epoch: 817, loss: 1.3105919361114502
epoch: 818, loss: 1.3101333379745483
epoch: 819, loss: 1.3096774816513062
epoch: 820, loss: 1.3092241287231445
epoch: 821, loss: 1.308773159980774
epoch: 822, loss: 1.3083246946334839
epoch: 823, loss: 1.3078784942626953
epoch: 824, loss: 1.3074346780776978
epoch: 825, loss: 1.3069936037063599
epoch: 826, loss: 1.3065544366836548
epoch: 827, 

In [72]:
shifts_pred

tensor([[ -3.4583,  -1.0995,  -0.1913,  ..., -11.5097, -14.5831, -17.9243],
        [ -3.4450,  -1.0836,  -0.1787,  ..., -11.3884, -14.4382, -17.7612],
        [ -3.4554,  -1.0850,  -0.1708,  ..., -11.3925, -14.4398, -17.8874],
        ...,
        [ -3.4883,  -1.1132,  -0.1923,  ..., -11.4141, -14.4596, -17.7866],
        [ -3.5228,  -1.1621,  -0.2488,  ..., -11.2006, -14.2407, -17.5608],
        [ -3.4557,  -1.0935,  -0.1879,  ..., -11.2472, -14.2974, -17.7299]],
       grad_fn=<LeakyReluBackward0>)

In [73]:
shifts

tensor([[ -2.8344,  -0.4313,   0.4293,  ..., -11.9093, -15.0479, -18.3271],
        [ -2.6625,  -0.2299,   0.6109,  ..., -11.3789, -14.4855, -17.8279],
        [ -2.7991,  -0.2632,   0.7154,  ..., -11.4353, -14.5021, -17.8652],
        ...,
        [ -3.2440,  -0.6030,   0.4282,  ..., -11.7126, -14.7787, -18.1606],
        [ -3.7102,  -1.3226,  -0.3567,  ...,  -8.7786, -11.7740, -15.0684],
        [ -2.8030,  -0.3818,   0.4941,  ...,  -9.4130, -12.5467, -15.8897]])

In [ ]:
def zmuv_normalize_phoneme_class(feature_matrix, calculate_mean_and_std=True, phoneme_mean=None, phoneme_std=None):
    '''
    return the normalized feature class of matrix phoneme_class_features_matrix, the mean and standard deviations
    inputs:
        feature_matrix (np.array): a 2d feature matrix of various instances of a phoneme. The phonemes instances are stacked on the z (?) axis
        calculate_mean_and_std (bool): whether to calculate mean and std or not. should be false if inputting values for phoneme_mean and phoneme_std
        phoneme_mean (np.array):  the average feature
        phoneme_std (np.array):  the standard deviation of features
        
    returns:
        normalized_features (np.array): the normalized feature matrix 
        phoneme_mean (np.array):  the average feature
        phoneme_std (np.array):  the standard deviation of features
    '''
    if calculate_mean_and_std:
        phoneme_mean, phoneme_std = np.mean(feature_matrix,0),np.std(feature_matrix,0)
    normalized_features = (feature_matrix - phoneme_mean)/(phoneme_std)
    
    return normalized_features, phoneme_mean, phoneme_std

In [ ]:
def get_phonemes_from_single_file(filename):
    '''
    this returns a dicitonary mapping phoneme names to numpy 1-D matrices of features of the phoneme class of a particular length (padded with nans if data is less than) found in a file
    inputs:
        filename (str): the input file
        
    '''
    output_dictionary = {}
    f = open(filename, "r")
    lines = f.readlines()
    
    f.close()
    for line in lines: 
        line = line.split()
        current_phoneme = line[-1]
        line = line[:-1]
        line = np.array(line,dtype="float64")
        if current_phoneme in output_dictionary:
            output_dictionary[current_phoneme].append(line)
        else:
            output_dictionary[current_phoneme] = [line]
        
    return output_dictionary

In [ ]:
# testing above function
TEST_FILE = 'Y:/personal/ojuba.mezisashe/Sentences/TIMIT_dp/output/TRAIN/DR1/FCJF0/SI648.txt'
print(TEST_FILE)
phonemes_from_file = get_phonemes_from_single_file(TEST_FILE)
# _ = get_phonemes_from_single_file(filename, 100) # edge case test

In [ ]:
phoneme = "aa"
phonemes_from_file[phoneme][0]

In [ ]:
a = np.array([1,2,3,4])
d = np.diag(a)

In [29]:
a = np.array([1,2,3,4])

In [30]:
a*a+a

array([ 2,  6, 12, 20])

## Combined Models

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric

# Define the PyTorch model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the PyTorch Geometric model
class GNN(torch_geometric.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, mlp):
        super(GNN, self).__init__()
        self.conv1 = torch_geometric.nn.GCNConv(input_size, hidden_size)
        self.conv2 = torch_geometric.nn.GCNConv(hidden_size, output_size)
        self.mlp = mlp
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = self.mlp(x)
        return x

# Define the combined model
class CombinedModel(nn.Module):
    def __init__(self, mlp, gnn):
        super(CombinedModel, self).__init__()
        self.mlp = mlp
        self.gnn = gnn
    
    def forward(self, x, edge_index):
        # Apply the MLP to the feature vectors
        x = self.mlp(x)
        # Apply the GNN to the graph and feature vectors
        x = self.gnn(x, edge_index)
        return x

# Create instances of the PyTorch and PyTorch Geometric models
mlp = MLP(input_size=10, hidden_size=20, output_size=5)
gnn = GNN(input_size=20, hidden_size=30, output_size=10, mlp=mlp)

# Create an instance of the combined model
combined_model = CombinedModel(mlp, gnn)